LLM은 직접 엑셀 표를 검색하거나 벡터화 불가

엑셀 데이터를 JSON 형식으로 구조화 -> 벡터화

In [2]:
# 식품대분류명 컬럼의 고유값(종류) 출력
import pandas as pd

EXCEL_PATH = r"C:\Users\thdwo\OneDrive\바탕 화면\프메 프젝\db\filtered_db.xlsx"
db = pd.read_excel(EXCEL_PATH)

print(db['식품대분류명'].unique())

['밥류' '빵 및 과자류' '면 및 만두류' '죽 및 스프류' '국 및 탕류' '찌개 및 전골류' '찜류' '구이류'
 '전·적 및 부침류' '볶음류' '조림류' '튀김류' '나물·숙채류' '생채·무침류' '김치류' '젓갈류' '장아찌·절임류'
 '음료 및 차류' '수·조·어·육류' '유제품류 및 빙과류']


카테고리 칼럼 추가

In [ ]:
import pandas as pd

# 카테고리 분류 함수 정의
def assign_category(row):
    category = row['식품대분류명']
    name = str(row['식품명'])  # 음식명에서 키워드 검색 시 사용

    if category == '밥류':
        return '밥'
    elif category in ['죽 및 스프류', '국 및 탕류', '찌개 및 전골류']:
        return '국'
    elif category in [
        '찜류', '구이류', '전·적 및 부침류', '볶음류', '조림류',
        '튀김류', '나물·숙채류', '생채·무침류', '김치류', '젓갈류',
        '장아찌·절임류', '수·조·어·육류'
    ]:
        return '반찬'
    elif category == '면 및 만두류':
        if '만두' in name:
            return '반찬'
        else:
            return '면'
    elif any(word in name.lower() for word in ['버거', '피자', '핫도그']):
        return '양식'
    elif category in ['빵 및 과자류', '음료 및 차류', '유제품류 및 빙과류']:
        return '디저트 & 빵'
    else:
        return '기타'

# 카테고리 열 채우기
df['카테고리'] = df.apply(assign_category, axis=1)

# 저장
df.to_excel(r"C:\Users\thdwo\OneDrive\바탕 화면\프메 프젝\db\filtered_db_add_cate.xlsx", index=False)

print("카테고리 열이 채워졌고, 파일이 저장되었습니다.")

RAG vectorstore 저장

In [1]:
# 🔧 FAISS 및 임베딩 로드 예시
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import pandas as pd
import json

# 🔗 파일 경로 설정
EXCEL_PATH = r"C:\Users\thdwo\OneDrive\바탕 화면\프메 프젝\db\filtered_db_add_cate.xlsx"
VECTORSTORE_DIR = "./vectorstore_json"

# ✅ 엑셀 파일 로드
df = pd.read_excel(EXCEL_PATH)

# 🚫 '영양성분함량기준량' 열 제외
if '영양성분함량기준량' in df.columns:
    df = df.drop(columns=['영양성분함량기준량'])

# 🔄 각 행을 JSON 형태로 변환
def row_to_json(row):
    return json.dumps(row.to_dict(), ensure_ascii=False)

# 📜 전체 문서 리스트 생성
docs = df.apply(row_to_json, axis=1).tolist()

# 🧠 임베딩 모델 로드
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 🗂️ FAISS 벡터스토어 생성
vectorstore = FAISS.from_texts(docs, embedding_model)

# 💾 벡터스토어 저장
vectorstore.save_local(VECTORSTORE_DIR)

print("✅ 벡터스토어 저장 완료!")


C:\Users\thdwo\AppData\Local\Temp\ipykernel_44572\3349078971.py:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


✅ 벡터스토어 저장 완료!
